In [1]:
# autoreload module
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# Import Network
import jax
from jax import numpy as jnp
from mtr.models.context_encoder.mtr_encoder import MTREncoder
from mtr.models.motion_decoder.bc_decoder import BCDecoder
from mtr.models.motion_decoder.q_decoder import QDecoder
from rl.actor import Actor
from rl.critic import Critic
from rl.encoder import Encoder
from rl.sac import SAC
import torch
import dataclasses
from waymax import config as _config
from waymax import datatypes
from waymax import dynamics
from rl_env.waymax_env import womd_loader, MultiAgentEnvironment
import tensorflow as tf


from mtr.config import cfg, cfg_from_yaml_file

2023-12-08 15:24:46.693566: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-08 15:24:46.693604: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-08 15:24:46.693608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/home/zixuzhang/mambaforge/envs/waymax/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonote

In [4]:
max_num_objects = 32

# create a dataset
WOMD_1_2_0_VAL_LOCAL = _config.DatasetConfig(
    path='/Data/Dataset/Waymo/V1_2_tf/validation_interactive/validation_interactive_tfexample.tfrecord@150',
    # path='/Data/Dataset/Waymo/V1_2_tf/validation/validation_tfexample.tfrecord@150',
    max_num_rg_points=30000,
    data_format=_config.DataFormat.TFRECORD,
    max_num_objects=max_num_objects,
    shuffle_seed = 0,
)

data_iter = womd_loader(data_config=WOMD_1_2_0_VAL_LOCAL)


# Config the multi-agent environment:
init_steps = 11

# Set the dynamics model the environment is using.
# Note each actor interacting with the environment needs to provide action
# compatible with this dynamics model.
dynamics_model = dynamics.DeltaLocal()

# Expect users to control all valid object in the scene.
env = MultiAgentEnvironment(
    dynamics_model=dynamics_model,
    config=dataclasses.replace(
        _config.EnvironmentConfig(),
        init_steps = init_steps,
        max_num_objects=max_num_objects,
        controlled_object=_config.ObjectType.MODELED,
        rewards = _config.LinearCombinationRewardConfig(
            rewards={
                'overlap': 1.0, # Positive is good.  
                'offroad': 1.0, # Negative is good.
                'kinematics': 1.0, # Negative is good.
            }
        )
    ),
)

In [5]:
with tf.device('/CPU:0'):
    _, cur_state = next(data_iter)


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


RuntimeError: Exiting device scope without proper scope nesting

In [ ]:
@jax.jit
def check_controlled(state: datatypes.SimulatorState):
    current_stamp = state.timestep
    is_valid = state.sim_trajectory.valid[..., current_stamp]
    
    is_modeled = jnp.logical_or(
        state.object_metadata.is_modeled,
        state.object_metadata.is_sdc
    )
    
    # is_modeled = is_valid
    # is_modeled.at[10:].set(False)
    is_vehicle = (state.object_metadata.object_types == 1)
    
    return jnp.logical_and(is_valid, jnp.logical_and(is_modeled, is_vehicle))

In [ ]:
cfg = cfg_from_yaml_file('tools/cfgs/waymo/rl_finetune.yaml', cfg)

model_dict = torch.load('output/bc_atten_4_unfreeze_state/epoch=19-step=1211340.ckpt', map_location=torch.device('cpu'))['state_dict']

In [ ]:


encoder_network = MTREncoder(cfg.MODEL.CONTEXT_ENCODER)
encoder_network.load_model(model_dict)

actor_network = BCDecoder(
    encoder_network.num_out_channels,
    cfg.MODEL.MOTION_DECODER
)
actor_network.load_model(model_dict)
critic_network = QDecoder(
    encoder_network.num_out_channels,
    cfg.MODEL.Q_DECODER
)

encoder = Encoder(
    model_cfg=cfg.MODEL.CONTEXT_ENCODER,
    is_controlled_func = check_controlled,
)

actor = Actor(
    cfg.SAC.ACTOR,
    actor_network
)
actor.eval()

critic = Critic(
    cfg.SAC.CRITIC,
    critic_network
)   


sac = SAC(
    None ,
    None,
    data_iter,
    env,
    encoder,
    actor,
    None,
    critic,
)

In [ ]:
sac.interact()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
